# LSTM Model Tuning-2

Secondly, to obtain the optimal restructured sequence length another tuning job done using AWS automatic model-tuning feature and Jupyter notebook associated to the task has include under [Appendix-F]. 

In [ ]:
import sagemaker
import boto3

import numpy as np                                # For performing matrix operations and numerical processing
import pandas as pd                               # For manipulating tabular data         
import os

region = boto3.Session().region_name    
smclient = boto3.Session().client('sagemaker')

bucket = '<Put Your bucket name>'
prefix = 'lstm-sankalpa-tuning'

s3 = sagemaker.Session()
role = sagemaker.get_execution_role()
print(str(s3))
print(role)
print(str(region))
print(str(smclient))

# Load pre-processed transmission data from AWS S3 bucket

In [ ]:
training_input_path   = s3.upload_data('datasets/train_restruct.npz', bucket = bucket, key_prefix=prefix+'/training')
validation_input_path = s3.upload_data('datasets/valid_restruct.npz', bucket = bucket, key_prefix=prefix+'/validation')
print(validation_input_path)

In [ ]:
custom_code_upload_location = 's3://'+bucket+'/'+prefix
model_artifact_location = 's3://'+bucket+'/'+prefix+'/artifacts'
print(custom_code_upload_location)
print(model_artifact_location)

In [ ]:
# !pygmentize lstm_model.py

# Local Mode Model Training 

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='lstm_model.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

# AWS Model Training Job

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='lstm_model.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          base_job_name='hpo-lstm-1'
                         )
# ml.t3.medium

# AWS Automatic Model Tunning Job

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs':        IntegerParameter(1, 50),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='Logarithmic'), 
    'batch-size':    IntegerParameter(32, 1024),
    'dense-layer':   IntegerParameter(5, 1024),
    'dropout':       ContinuousParameter(0.0, 0.9)
    'sequence-length': IntegerParameter(2, 25),
}

objective_metric_name = 'val_acc'
objective_type = 'Maximize'
metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=10,
                            max_parallel_jobs=1,
                            objective_type=objective_type)

In [ ]:
tuner.fit({'training': training_input_path, 'validation': validation_input_path})